In [1]:
SERIES_LOCATION = '/home/musslebot/Documents/SERTEST/'
SECTION_NUMBER = 64

In [2]:
import os
import sys

import numpy
from PyQt5 import QtCore, QtGui, QtWidgets
from skimage import io
from skimage.transform import warp

from pyrecon.tools.reconstruct_reader import process_series_directory

app = QtWidgets.QApplication(sys.argv)

In [3]:
# Load XML files into Python
series = process_series_directory(SERIES_LOCATION)
section = series.sections[SECTION_NUMBER]
image = section.images[0]
image_path = os.path.join(SERIES_LOCATION, image.src)
image_transform = image.transform._tform
contour = section.contours[0]

In [4]:
# Load image into memory
img = io.imread(image_path, as_grey=True)
t_img = warp(img, image_transform)
t_img = (t_img * 255).round().astype('uint8')
l, w = len(t_img), len(t_img[0])
qimage = QtGui.QImage(t_img.tobytes(), l, w, QtGui.QImage.Format_Grayscale8)
pixmap = QtGui.QPixmap(qimage)

In [7]:
# Transform contour
# - Scale contour by image mag
contour.points = list(map(tuple, numpy.asarray(contour.points)/image.mag))
# - Apply transform to contour
contour.points = list(map(tuple, 
    contour.transform._tform.inverse(numpy.asarray(contour.points))
))

translation_vector = numpy.array([0, pixmap.size().height()])
flip_vector = numpy.array([1, -1])
contour.points = list(map(tuple, 
    translation_vector + (numpy.array(list(contour.shape.exterior.coords))*flip_vector)
))

In [8]:
# Draw polygon
polygon = QtGui.QPolygon()
for p in contour.points:
    polygon.append(QtCore.QPoint(*p))
painter = QtGui.QPainter()
painter.begin(pixmap)
pen = QtGui.QPen()
pen.setColor(QtGui.QColor('red'))
pen.setWidth(15)
painter.setPen(pen)
painter.drawConvexPolygon(polygon)
painter.end()

True

In [9]:
# Crop image
minx, miny, maxx, maxy = contour.shape.bounds
x = minx - 200
y = miny - 200
width = maxx - x + 300
height = maxy - y + 300
rect = [x, y, width, height]
pixmap = pixmap.copy(*rect)
pixmap = pixmap.scaled(500, 500, QtCore.Qt.KeepAspectRatio)
label = QtWidgets.QLabel()
label.setPixmap(pixmap)
label.show()

In [10]:
from IPython.display import Image
Image(img)